In [1]:
from bluepy.btle import Scanner, DefaultDelegate

class ScanDelegate(DefaultDelegate):
    def __init__(self):
        DefaultDelegate.__init__(self)

    def handleDiscovery(self, dev, isNewDev, isNewData):
        if isNewDev:
            print("Discovered device ",  dev.addr)
        elif isNewData:
            print("Received new data from", dev.addr)

scanner = Scanner().withDelegate(ScanDelegate())
devices = scanner.scan(10.0)

for dev in devices:
    print("Device %s (%s), RSSI=%d dB" % (dev.addr, dev.addrType, dev.rssi) )
    for (adtype, desc, value) in dev.getScanData():
        print("  %s = %s" % (desc, value) )

Discovered device  54:6c:0e:b1:5d:73
Discovered device  54:6c:0e:b1:5d:5e
Discovered device  63:66:11:7b:96:71
Discovered device  26:89:6a:30:2a:03
Discovered device  33:b5:34:68:c3:21
Discovered device  54:6c:0e:b1:5e:c0
Discovered device  3a:fc:f4:2a:f3:d3
Discovered device  54:6c:0e:b1:58:3f
Discovered device  14:ef:28:e2:29:6a
Discovered device  47:40:e7:2b:7d:72
Discovered device  1e:73:81:a6:33:44
Received new data from 54:6c:0e:b1:58:3f
Received new data from 54:6c:0e:b1:5d:73
Received new data from 54:6c:0e:b1:5d:5e
Discovered device  54:6c:0e:b1:5e:de
Discovered device  44:81:2d:4a:b5:72
Discovered device  39:d9:d0:d5:bd:2b
Received new data from 54:6c:0e:b1:5e:c0
Discovered device  58:2d:34:30:36:d1
Discovered device  2c:41:a1:98:74:f5
Device 54:6c:0e:b1:58:3f (public), RSSI=-46 dB
  Flags = 06
  Incomplete 16b Services = 0000180d-0000-1000-8000-00805f9b34fb
  Tx Power = 00
  0x12 = 50002003
  Complete Local Name = HRScope
Device 39:d9:d0:d5:bd:2b (random), RSSI=-73 dB
  Manu

In [26]:
from bluepy import btle
import re
import time
class EEGBand(object):
    def __init__(self):
        self.quality = None
        self.attention = None
        self.meditation = None
        self.blinkStrength = None
        self.timestamp = None
        self.received = False
        self.count = 0

    def handleNotification(self, cHandle, data):
        print(cHandle,data)
        
        if cHandle == 32:
            index = 1
            flag = data[0]
            if flag & 1 == 1:
                self.quality = data[index]
                index = index + 1
            if flag & 2 == 2:
                self.attention = data[index]
                index = index + 1
            if flag & 4 == 4:
                self.meditation = data[index]
                index = index + 1
            if flag & 8 == 6:
                self.blinkStrength = data[index]
                index = index + 1
            if flag & 16 == 16:
                self.timestamp = data[index:]
                index = index + 1
            print(flag,self.quality,self.attention,self.meditation,self.blinkStrength,self.timestamp)
            self.count = self.count + 1
            if(self.count > 4): self.received = True
def get_data():
    address = "54:6c:0e:b1:5e:de"
    p = btle.Peripheral(address)
    delegate = EEGBand()
    p.withDelegate( delegate )
    print(p.readCharacteristic(0x20))  
    print(p.readCharacteristic(0x21))
    p.writeCharacteristic(0x21, bytearray([1, 0]), True)
    print(p.readCharacteristic(0x20))    
    print(p.readCharacteristic(0x21))
#     p.writeCharacteristic(0x25, bytearray([1, 0]), True)
#     p.writeCharacteristic(0x29, bytearray([1, 0]), True)

    while not delegate.received:
        p.waitForNotifications(5.0)
    p.disconnect()
    return
    temperature = delegate.temperature
    humidity = delegate.humidity
    print("温度：",temperature,"  湿度：",humidity,"%")
get_data()

b'\x00'
b'\x00\x00'
b'\x00'
b'\x01\x00'
32 b'\x17\xc8\x00\x00\x03\x00\x00\x00'
23 200 0 0 None b'\x03\x00\x00\x00'
32 b'\x17\xc8\x00\x00\x04\x00\x00\x00'
23 200 0 0 None b'\x04\x00\x00\x00'
32 b'\x17\xc8\x00\x00\x05\x00\x00\x00'
23 200 0 0 None b'\x05\x00\x00\x00'
32 b'\x17\xc8\x00\x00\x06\x00\x00\x00'
23 200 0 0 None b'\x06\x00\x00\x00'
32 b'\x17\xc8\x00\x00\x07\x00\x00\x00'
23 200 0 0 None b'\x07\x00\x00\x00'
